# Aufgabe


In dem Aufgaben finden Sie 2 Dateien. Die Dateien enthalten englische Reviews von Filmen. Die eine Datei enthält negative die andere positive Reviews. 

Wir wollen einen Klassifikator bauen, der vorhersagt, ob ein Review positiv oder negativ ist. Machen Sie dazu folgendes:

1. Teilen Sie die Daten in Test und Trainingsdaten ein.
2. Extrahieren Sie alle Adjektive, Verben und Adverbien aus den Reviews
3. Nutzen Sie alle Adjektive, Verben und Adverbien, die mindesten 5 Mal vorkommen als Merkmale und Trainieren Sie einen Klassifikator, der logistische Regression nutzt.
5. Evaluieren Sie den Klassifikator

# Lösung

In [20]:
import nltk
import re

from nltk.corpus import wordnet as wn
lemmatizer = nltk.WordNetLemmatizer()

from collections import Counter

stopwords = nltk.corpus.stopwords.words('english')
 
def wntag(pttag):
    if pttag in ['JJ', 'JJR', 'JJS']:
        return wn.ADJ
    elif pttag in ['RB', 'RBR', 'RBS']:
        return wn.ADV
    elif pttag in ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']:
        return wn.VERB
    return None

def lemmatize(lemmatizer,word,pos):
    
    if pos == None:
        return None
    else:  
        return lemmatizer.lemmatize(word,pos)

def features_from_text(text):
    wordcounts = Counter()
    tlen = 0
    
    satzliste =  nltk.sent_tokenize(text,language='english')
    
    for satz in satzliste:
        tokens =  nltk.word_tokenize(satz,language='english')            
        tokens = [lemmatize(lemmatizer, lemma, wntag(pos))  for (word,pos) in nltk.pos_tag(tokens)]
        tokens = [lemma for lemma in tokens if lemma!=None]
        tokens = [t for t in tokens if t.lower() not in stopwords]
        tokens = [t for t in tokens if re.search('^\w+$',t)]
        tlen += len(tokens)
        wordcounts.update(tokens)
 
    return {w:wordcounts[w]/tlen for w in wordcounts}

In [21]:
import codecs 

def readtext(dateiname):
    text = ''
    d = codecs.open(dateiname,'r','utf8')
    for zeile in d:
        text += str(zeile)
    d.close()

    return text

In [22]:
import glob

def read_data(directories):
    docs = []
    for directory in directories:
        for file in glob.glob(directory+"/*.txt"):
            text = readtext(file)
            docs.append((features_from_text(text),directory))

    return docs

data = read_data(['neg','pos'])

In [23]:
import random

random.shuffle(data)
train_data = data[:1600]
test_data = data[1600:]

In [24]:
docfreq = Counter()
for (wfreq,c) in train_data:
    docfreq.update(wfreq.keys())

In [25]:
docfreq.most_common(10)

[]

In [26]:
from sklearn import linear_model, datasets


allfeatures = [w for w in docfreq if docfreq[w] > 5]

def make_feat_vec(featmap,featlist):
    vec = []
    for f in featlist:
        vec.append(featmap.get(f,0.0))
    return vec

train_vec =  [make_feat_vec(feats,allfeatures) for feats,cls in train_data]
train_label = [cls for feats,cls in train_data]

In [27]:
logreg = linear_model.LogisticRegression(C=1e9,verbose=True)
logreg.fit(train_vec,train_label)

ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [28]:
v = make_feat_vec(test_data[2][0],allfeatures) 
logreg.predict([v])

IndexError: list index out of range

In [29]:
test_data[2][1]

IndexError: list index out of range

In [30]:
test_vec = [make_feat_vec(feats,allfeatures) for feats,cls in test_data]
test_label = [cls for feats,cls in test_data]

pred_label = list(logreg.predict(test_vec))

NotFittedError: This LogisticRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [31]:
correct = 0
for i in range(len(test_label)):
    if test_label[i] == pred_label[i]:
        correct+=1
print("{0:.1f} Prozent korrekt".format(100* float(correct)/len(test_label)))

ZeroDivisionError: float division by zero

In [32]:
cm = nltk.ConfusionMatrix(test_label, pred_label)
print(cm)

NameError: name 'pred_label' is not defined